In [39]:
import cv2 
import numpy as np 
cap = cv2.VideoCapture(r'C:\Users\student\Downloads\satrain.mp4') 
if not cap.isOpened(): 
   raise ValueError("Error: Could not open video file.") 
frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4)) 
fps = int(cap.get(cv2.CAP_PROP_FPS)) 
out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height)) 
ret, old_frame = cap.read() 
if not ret: 
   raise ValueError("Error: Could not read the first frame.") 
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY) 
lk_params = dict(winSize=(15, 15), maxLevel=2, 
                criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)) 
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7) 
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params) 
if p0 is None or len(p0) == 0: 
   raise ValueError("Error: No features detected in the first frame.") 
p0 = np.array(p0, dtype=np.float32) 
mask = np.zeros_like(old_frame) 
while cap.isOpened(): 
   ret, frame = cap.read() 
   if not ret: 
       break 
   frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
   if p0 is not None and len(p0) > 0: 
       p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params) 
        
       if p1 is not None and st is not None: 
           good_new = p1[st == 1] 
           good_old = p0[st == 1] 
           for i, (new, old) in enumerate(zip(good_new, good_old)): 
               a, b = map(int, new.ravel()) 
               c, d = map(int, old.ravel()) 
               mask = cv2.line(mask, (c, d), (a, b), (0,0 , 255), 2) 
               frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1) 
           p0 = good_new.reshape(-1, 1, 2) 
   if p0 is None or len(p0) < 10: 
       p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params) 
       if p0 is not None: 
           p0 = np.array(p0, dtype=np.float32) 
   output = cv2.add(frame, mask) 
   out.write(output) 
   old_gray = frame_gray.copy() 
cap.release() 
out.release() 
cv2.destroyAllWindows() 

In [43]:
import cv2
import numpy as np

def lucas_kanade_optical_flow(video_path, output_path='outputup.mp4'):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Error: Could not open video file.")

    # Video properties
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    # Read first frame
    ret, old_frame = cap.read()
    if not ret:
        raise ValueError("Error: Could not read the first frame.")

    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    # Lucas-Kanade Parameters
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)

    # Detect Initial Features
    p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)
    if p0 is None:
        raise ValueError("Error: No features detected in the first frame.")

    mask = np.zeros_like(old_frame)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Compute Optical Flow
        p1, st, _ = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        if p1 is not None and st is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            for new, old in zip(good_new, good_old):
                a, b = map(int, new.ravel())
                c, d = map(int, old.ravel())
                mask = cv2.line(mask, (c, d), (a, b), (0, 0, 255), 2)
                frame = cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)

            p0 = good_new.reshape(-1, 1, 2)

        # If too few points, detect new features
        if p0 is None or len(p0) < 10:
            p0 = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)
            if p0 is not None:
                p0 = np.array(p0, dtype=np.float32)

        # Merge mask with the frame
        output = cv2.add(frame, mask)
        out.write(output)

        old_gray = frame_gray.copy()

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Run the function
lucas_kanade_optical_flow(r'C:\Users\student\Downloads\satrain.mp4')
